# Test of GraphSAGE
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [2]:
import os
import dgl
import csv
import json
import torch
import random
import subprocess
import torch as th
import numpy as np
import pandas as pd
import torch.nn as nn
import dgl.nn as dglnn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from torch.optim import AdamW, lr_scheduler
from dgl.nn import GraphConv, GATConv, SAGEConv
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup

os.environ['CUDA_VISIBLE_DEVICES'] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- check the GPU and assign the GPU by the best memory usage

In [3]:
# def get_free_gpu():
#     try:
#         # Run nvidia-smi command to get GPU details
#         _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
#         command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
#         memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
#         memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
#         # Get the GPU with the maximum free memory
#         best_gpu_id = memory_free_values.index(max(memory_free_values))
#         return best_gpu_id
#     except:
#         # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
#         return 0

# if torch.cuda.is_available():
#     # Get the best GPU ID based on free memory and set it
#     best_gpu_id = get_free_gpu()
#     device = torch.device(f"cuda:{best_gpu_id}")
# else:
#     device = torch.device("cpu")
#     print("there's no available GPU")

# # device = torch.device(f"cuda:{1}")
# print(device)


## Fix the seed

In [4]:
#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Load the embedding

In [5]:
DIM = 256
embedding = 'transE'

embedding = f'{embedding}_{DIM}'
with open(f"../../data/4_embedding/{embedding}.vec.json", "r") as f:
    tmp = json.load(f)

index2entemb = {idx:emb for idx, emb in enumerate(tmp["ent_embeddings.weight"])}
index2relemb = {idx:emb for idx, emb in enumerate(tmp["rel_embeddings.weight"])}

In [6]:
len(index2entemb)

824642

In [7]:
len(index2relemb)

27

- this file is 55 GB -> takes about 1 min to load it

In [8]:
with open("../../data/all_graph_data.jsonl", "r") as f:
    print("Loading the data...")
#     input_data = list(f)
#     input_data = [json.loads(line) for idx, line in tqdm(f, desc="Loading")]

    input_data = []
    for idx, line in tqdm(enumerate(f), total=16900, desc="Loading"):
        input_data.append(json.loads(line))
        
    print("FINISH...")

Loading the data...


Loading:   0%|          | 0/16900 [00:00<?, ?it/s]

FINISH...


In [9]:
len(input_data)

16900

- Convert the 'node_feat' and 'edge_attr' from int to embedding
    - takes about 45 min to transform the embedding
    - if use original method -> takes about 60 hours

In [10]:
type(input_data)

list

In [11]:
type(input_data[0])

dict

- need to get the new graph.jsonl -> the id is not corresponding

In [13]:
# ============ If type(input_data[0] == dict) ============
for data_point in tqdm(input_data):
    data_point['node_feat'] = [index2entemb[node_id] for node_id in data_point['node_feat']]
    data_point['edge_attr'] = [index2relemb[edge_id] for edge_id in data_point['edge_attr']]

  0%|          | 0/16900 [00:00<?, ?it/s]

TypeError: unhashable type: 'list'

In [ ]:
type(input_data[0])

In [14]:
len(input_data[1]['node_feat'][0])

256

## Data Loader

In [15]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]
        return data

def collate(samples):
    data_list = samples
    batched_graphs = []
    for data in data_list:
        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"])

        g.ndata['feat'] = th.tensor(data["node_feat"])
        g.edata['feat'] = th.tensor(data["edge_attr"])
        g.edata['label'] = th.tensor(data["labels"])  # Add edge labels to graph

        batched_graphs.append(g)
    
    return dgl.batch(batched_graphs)

In [16]:
# split 8:1:1 (train, valid, test)
train_data, test_data = train_test_split(input_data, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)


dataset_data = {
    'train': GraphDataset(train_data, device),
    'valid': GraphDataset(valid_data, device),
    'test': GraphDataset(test_data, device)
}

print("Datasets loaded and ready for training!")

Datasets loaded and ready for training!


- choose batch size

In [17]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(8)

- Turn the print message to a log file

In [18]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"./log_message/{formatted_time}_GraphSAGE_transE_256.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

./log_message/0204_07:01_GraphSAGE_transE_256.log


### Model

In [19]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')
        self.dropout = nn.Dropout(0.25)

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h)
        return h

In [20]:
class MLPPredictor(nn.Module):
    def __init__(self, out_feats, out_classes):
        super().__init__()
        self.W = nn.Linear(out_feats*2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(torch.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [21]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_classes):
        super().__init__()
        self.sage = GraphSAGE(in_features, hidden_features, out_features)
        self.pred = MLPPredictor(out_features, num_classes)
      
    def forward(self, g, node_feat, return_logits=False):
        h = self.sage(g, node_feat)
        logits = self.pred(g, h)
        
        return logits

- Model Forward  

In [22]:
def model_fn(batched_g, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
#     batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = batched_g.edata['label'].to(device)
    
    logits = model(batched_g, batched_g.ndata['feat'].float())

    loss = criterion(logits, labels)

    output = torch.softmax(logits, dim=1)
    preds = output.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
        
    return loss, accuracy, preds

In [23]:
import re

def build_dictionary(file_path):
    with open(file_path, 'r') as file:
        next(file)
        # 使用正则表达式去除行末的数字
        dictionary = {re.sub(r'\s\d+$', '', line.strip()): index for index, line in enumerate(file)}
    return dictionary
    
file_path = '../../data/3_openKE/label2id.txt'  # 替換為您檔案的路徑
label2index = build_dictionary(file_path)
index2label = {v: k for k, v in label2index.items()}

index2label

{0: 'T1059.001_702bfdd2-9947-4eda-b551-c3a1ea9a59a2_B',
 1: 'T1078.001_d0ca00832890baa1d42322cf70fcab1a_B',
 2: 'T1074.001_e6dfc7e89359ac6fa6de84b0e1d5762e_B',
 3: 'T1491_68235976-2404-42a8-9105-68230cfef562_B',
 4: 'T1016_14a21534-350f-4d83-9dd7-3c56b93a0c17_B',
 5: 'T1491_47d08617-5ce1-424a-8cc5-c9c978ce6bf9_I',
 6: 'T1074.001_4e97e699-93d7-4040-b5a3-2e906a58199e_I',
 7: 'T1040_6881a4589710d53f0c146e91db513f01_B',
 8: 'T1547.009_b6e5c895c6709fe289352ee23f062229_B',
 9: 'T1564.001_66a5fd5f244819181f074dd082a28905_B',
 10: 'T1047_f4b0b4129560ea66f9751275e82f6bab_B',
 11: 'T1112_257313a3c93e3bb7dfb60d6753b09e34_I',
 12: 'T1047_ac2764f7a67a9ce92b54e8e59b361838_B',
 13: 'T1518.001_33a24ff44719e6ac0614b58f8c9a7c72_B',
 14: 'T1204.002_522f3f35cd013e63830fa555495a0081_I',
 15: 'T1059.001_ccdb8caf-c69e-424b-b930-551969450c57_B',
 16: 'T1105_0856c235a1d26113d4f2d92e39c9a9f8_B',
 17: 'T1547_fe9eeee9a7b339089e5fa634b08522c1_I',
 18: 'T1574.001_63bbedafba2f541552ac3579e9e3737b_B',
 19: 'T1137.002

In [24]:
type(index2entemb)

dict

### Main Training Loop

- For release the GPU memory
    - no need to restart the kernel

In [25]:
# # For release the GPU memory
# # No need to restart the kernel

# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [26]:
seed = 5269
in_dim = DIM # dimension of the node feature
hidden_dim = 512
out_dim = 1024
num_classes = len(label2index)

lr = 5e-4

total_steps = 100
patience = 10
waiting = 0

In [27]:
model = Model(in_dim, hidden_dim, out_dim, num_classes)
best_model_path = f"./checkpoint_graphSAGE/best_model_GraphSAGE_{embedding}-small_batchsize-bigdim.pt"

optimizer = AdamW(model.parameters(), lr)

scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0, last_epoch=-1, verbose=False)

criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.BCEWithLogitsLoss()

In [28]:
same_seeds(seed)
model = model.to(device)
best_val_loss = float('inf')

# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for batched_g in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

    # scheduler affect the performance a lot
    # scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for batched_g in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)
 
    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("============================== Early stopping ==================================")
            break

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/1690 [00:00<?, ?it/s]

02/04/2024, 07:03:56# total batches: 1690
02/04/2024, 07:03:56# Epoch 0 | Train Loss: 1.2901 | Train Accuracy: 0.7609


Validation:   0%|          | 0/212 [00:00<?, ?it/s]

02/04/2024, 07:04:13# Validation Loss: 0.9023 | Validation Accuracy: 0.8133

02/04/2024, 07:04:13# Find a better model!!


...

02/04/2024, 09:13:21# Validation Loss: 0.1475 | Validation Accuracy: 0.9644



Training:   0%|          | 0/1690 [00:00<?, ?it/s]

02/04/2024, 09:16:02# total batches: 1690
02/04/2024, 09:16:02# Epoch 45 | Train Loss: 0.0566 | Train Accuracy: 0.9839


Validation:   0%|          | 0/212 [00:00<?, ?it/s]

02/04/2024, 09:16:17# Validation Loss: 0.1713 | Validation Accuracy: 0.9596

02/04/2024, 09:16:17# ============================== Early stopping ==================================


### test of valid and test part is ``graph``

- 60 APs in training x 10000times
- 5 APs in validation x 4 times
- 3 APs in test x 4 times
- Batch size = 4

In [29]:
# load the pretrained model
# pretrained_model_path = './checkpoint_graphSAGE/best_model_GraphSAGE_transE_50.pt'
model.load_state_dict(torch.load(best_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, count, which_type='test')
        labels = batched_g.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')

Testing:   0%|          | 0/212 [00:00<?, ?it/s]

02/04/2024, 09:16:18# labels: tensor([248, 248, 248,  ..., 248, 248, 248], device='cuda:0') torch.Size([4711])
02/04/2024, 09:16:18# predicted: tensor([248, 248, 248,  ..., 248, 248, 248], device='cuda:0') torch.Size([4711])
02/04/2024, 09:16:36# Test Accuracy: 96.7539847233331 %





In [30]:
report_data = classification_report(true_labels, predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

output_path = "./result"
if not os.path.isdir(output_path):
    os.makedirs(output_path)
    
report_df.reset_index(inplace=True, names='label')

label_list = []
for idx, row in report_df.iterrows():
    if row["label"].isdigit():
        row["label"] = index2label[int(row["label"])]
    label_list.append(row["label"])
report_df["label"] = label_list

report_df.to_csv(f'{output_path}/result_{embedding}-small_batchsize-bigdim-2.csv', index=False)
print("report output at: ", f'{output_path}/result_{embedding}.csv')

report_df

/media/Raid6_disk/bai/anaconda3/envs/DGL/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


report output at:  ./result/result_transE_256.csv


/media/Raid6_disk/bai/anaconda3/envs/DGL/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/media/Raid6_disk/bai/anaconda3/envs/DGL/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,label,precision,recall,f1-score,support
0,T1059.001_702bfdd2-9947-4eda-b551-c3a1ea9a59a2_B,0.041667,0.062500,0.050000,16.00000
1,T1078.001_d0ca00832890baa1d42322cf70fcab1a_B,1.000000,1.000000,1.000000,13.00000
2,T1074.001_e6dfc7e89359ac6fa6de84b0e1d5762e_B,0.000000,0.000000,0.000000,25.00000
3,T1491_68235976-2404-42a8-9105-68230cfef562_B,0.000000,0.000000,0.000000,18.00000
4,T1016_14a21534-350f-4d83-9dd7-3c56b93a0c17_B,1.000000,1.000000,1.000000,27.00000
...,...,...,...,...,...
276,T1003.003_9f73269695e54311dd61dc68940fb3e1_B,1.000000,1.000000,1.000000,13.00000
277,T1547.001_163b023f43aba758d36f524d146cb8ea_B,0.285714,0.080000,0.125000,25.00000
278,accuracy,0.967540,0.967540,0.967540,0.96754
279,macro avg,0.637537,0.643151,0.628806,813921.00000


### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [36]:
seed = 8787
same_seeds(seed)

model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [69]:
# model.layer1.fc_self.weight
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[ 0.0181, -0.0857,  0.1973,  ...,  0.2417,  0.2702, -0.3041],
        [-0.0768, -0.2723, -0.2001,  ...,  0.2989, -0.1387, -0.1940],
        [ 0.2582, -0.0822,  0.3086,  ..., -0.0257, -0.1119, -0.0335],
        ...,
        [ 0.2274, -0.0411, -0.0334,  ..., -0.1679,  0.2455,  0.2424],
        [ 0.1375,  0.2813,  0.0775,  ...,  0.1337,  0.2065,  0.2618],
        [-0.0951,  0.1010, -0.2586,  ..., -0.1242, -0.0631,  0.0924]],
       requires_grad=True)

- Check if model really load the model_dict

In [70]:
model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[ 0.0181, -0.0857,  0.1973,  ...,  0.2417,  0.2702, -0.3041],
        [-0.0768, -0.2723, -0.2001,  ...,  0.2989, -0.1387, -0.1940],
        [ 0.2582, -0.0822,  0.3086,  ..., -0.0257, -0.1119, -0.0335],
        ...,
        [ 0.2274, -0.0411, -0.0334,  ..., -0.1679,  0.2455,  0.2424],
        [ 0.1375,  0.2813,  0.0775,  ...,  0.1337,  0.2065,  0.2618],
        [-0.0951,  0.1010, -0.2586,  ..., -0.1242, -0.0631,  0.0924]],
       requires_grad=True)